In [51]:
import numpy as np
import pandas as pd
from ChannelAttribution import *
import plotly.io as pio

# START

In [52]:
#!pip install ChannelAttribution

In [53]:
# Read data

In [54]:
Data = pd.read_csv("Data.csv", sep=";")

In [55]:
Data.head()

,path,total_conversions,total_conversion_value,total_null
0,eta > iota > alpha > eta,1,0.244,3
1,iota > iota > iota > iota,2,3.195,6
2,alpha > iota > alpha > alpha > alpha > iota > ...,2,6.754,6
3,beta > eta,1,2.402,3
4,iota > eta > theta > lambda > lambda > theta >...,0,0.000,2


# heuristic_models

In [56]:
H=heuristic_models(Data,"path","total_conversions",var_value="total_conversion_value")

*** Looking to run more advanced attribution? Try ChannelAttribution Pro for free! Visit https://channelattribution.io/product


In [57]:
H.head()

,channel_name,first_touch_conversions,first_touch_value,last_touch_conversions,last_touch_value,linear_touch_conversions,linear_touch_value
0,eta,3164.0,11909.476213,4167.0,16754.203800,3539.951157,13783.497051
1,iota,4606.0,19597.261277,3355.0,13487.974271,3857.096221,15988.988995
2,alpha,6308.0,19121.272356,8447.0,28414.214274,7574.718594,24524.709569
3,beta,2831.0,12235.591744,989.0,3850.020987,2083.500145,8954.266717
4,theta,1606.0,6652.349347,653.0,2799.091988,1022.801394,4295.743619


# markov_model

In [58]:
M=markov_model(Data, "path", "total_conversions", var_value="total_conversion_value")

Number of simulations: 100000 - Convergence reached: 1.46% < 5.00%
Percentage of simulated paths that successfully end before maximum number of steps (46) is reached: 99.99%
*** Looking to run more advanced attribution? Try ChannelAttribution Pro for free! Visit https://channelattribution.io/product


In [59]:
M.head()

,channel_name,total_conversions,total_conversion_value
0,eta,3338.742007,12955.860362
1,iota,3909.928296,15194.374336
2,alpha,5272.775061,18723.710508
3,beta,2579.641010,9876.446884
4,theta,1955.429283,7445.351252


# Now we can merge attributed conversion for each method:

In [60]:
R=pd.merge(H,M,on="channel_name",how="inner")
R.head()

,channel_name,first_touch_conversions,first_touch_value,last_touch_conversions,last_touch_value,linear_touch_conversions,linear_touch_value,total_conversions,total_conversion_value
0,eta,3164.0,11909.476213,4167.0,16754.203800,3539.951157,13783.497051,3338.742007,12955.860362
1,iota,4606.0,19597.261277,3355.0,13487.974271,3857.096221,15988.988995,3909.928296,15194.374336
2,alpha,6308.0,19121.272356,8447.0,28414.214274,7574.718594,24524.709569,5272.775061,18723.710508
3,beta,2831.0,12235.591744,989.0,3850.020987,2083.500145,8954.266717,2579.641010,9876.446884
4,theta,1606.0,6652.349347,653.0,2799.091988,1022.801394,4295.743619,1955.429283,7445.351252


In [61]:
# Selecting only the conversions
R1=R[["channel_name","first_touch_conversions","last_touch_conversions","linear_touch_conversions","total_conversions"]]

In [62]:
# Renaming columns
R1.columns=["channel_name","first_touch","last_touch","linear_touch","markov_model"]

In [63]:
R1.head()

,channel_name,first_touch,last_touch,linear_touch,markov_model
0,eta,3164.0,4167.0,3539.951157,3338.742007
1,iota,4606.0,3355.0,3857.096221,3909.928296
2,alpha,6308.0,8447.0,7574.718594,5272.775061
3,beta,2831.0,989.0,2083.500145,2579.641010
4,theta,1606.0,653.0,1022.801394,1955.429283


In [64]:
# unpivot a DataFrame from a wide to a long format. It essentially converts columns into rows
R1=pd.melt(R1, id_vars="channel_name")
R1.head(20)

,channel_name,variable,value
0,eta,first_touch,3164.0
1,iota,first_touch,4606.0
2,alpha,first_touch,6308.0
3,beta,first_touch,2831.0
4,theta,first_touch,1606.0
5,lambda,first_touch,902.0
6,kappa,first_touch,74.0
7,zeta,first_touch,27.0
8,epsilon,first_touch,99.0
9,gamma,first_touch,165.0


In [65]:
# Data prep for visulization

In [66]:
data = [dict(type = "histogram",histfunc="sum",x = R1.channel_name,y = R1.value, 
    transforms = [dict(
    type = "groupby",
    groups = R1.variable,
  )],
)]


In [67]:
fig = dict({"data":data})
pio.show(fig,validate=False)

#  Finished

In [68]:
# We can also get the same plot in terms of attributed conversion value:

In [69]:
R2=R[["channel_name","first_touch_value","last_touch_value",\
"linear_touch_value","total_conversion_value"]]
R2.columns=["channel_name","first_touch","last_touch","linear_touch","markov_model"]

R2=pd.melt(R2, id_vars="channel_name")

data = [dict(
  type = "histogram",
  histfunc="sum",
  x = R2.channel_name,
  y = R2.value,
  transforms = [dict(
    type = "groupby",
    groups = R2.variable,
  )],
)]


In [70]:
fig = dict({"data":data})
pio.show(fig,validate=False)